In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

training_set = [[0,0],
[0,0],
[0,1.01],
[0,0],
[1.01,0],
[0,0],
[0,0],
[0,2.02],
[0,0],
[2.02,0],
[0,0],
[0,0],
[0,0],
[0,1.12],
[0,0],
[1.12,0],
[0,0],
[0,0],
[0,1.03],
[0,0],
[1.03,0],
[0,0],
[0,0],
[0,1.11],
[0,0],
[1.11,0],
[0,0],
[0,2.14],
[0,0],
[2.14,0],
[0,0],
[0,1.07],
[0,0],
[1.07,0],
[0,0],
[0,0],
[0,0],
[0,3.03],
[0,0],
[3.03,0],
[0,0],
[0,0],
[0,0],
[0,1.18],
[0,0],
[1.18,0],
[0,0],
[0,2.01],
[0,0],
[2.01,0],
[0,1.05],
[0,0],
[1.05,0],
[0,0],
[0,1],
[0,0],
[1,1.1],
[0,0],
[1.1,0]]


In [3]:
learning_rate = 0.005
seq_length = 3 #sequence length
input_size = 2 #number of features
hidden_size = 4
num_classes = 2
num_classes_out = 1
num_layers = 1
num_epochs = 500

clip = 10

def sliding_windows(data, seq_length):
    x = []
    y = []
    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)
    return np.array(x),np.array(y)

training_data = training_set
x, y = sliding_windows(training_data, seq_length)

train_size = 48
test_size = len(y) - train_size
trainX = torch.Tensor(np.array(x[0:train_size]))
trainX = Variable(trainX)
testX = torch.Tensor(np.array(x[train_size:len(x)]))
testX = Variable(testX)
trainY = torch.Tensor(np.array(y[0:train_size][:,0]))
trainY = Variable(trainY)
testY = torch.Tensor(np.array(y[train_size:len(y)][:,0])) #select first column by [:,0] for many to one
testY = Variable(testY)

In [4]:

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,dropout=0.25)
#         self.selu = nn.RELU()
        self.fc = nn.Linear(hidden_size, num_classes_out)

    def forward(self, x):
        batch_size = x.size(0)
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.contiguous().view(-1, self.hidden_size)
        out = self.fc(h_out)
        out = out.view(batch_size, -1)
        out = out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        # return out, hidden
        return out






lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
criterion = torch.nn.MSELoss()
# criterion = torch.nn.SmoothL1Loss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    lstm.zero_grad()
    outputs = lstm(trainX)
    optimizer.zero_grad()
    loss = criterion(outputs, trainY)
    loss.backward()
    nn.utils.clip_grad_norm_(lstm.parameters(), clip)
    optimizer.step()
    if epoch%50==0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

lstm.eval()


/Users/ahemf/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 0, loss: 0.75714
Epoch: 50, loss: 0.38298
Epoch: 100, loss: 0.24144
Epoch: 150, loss: 0.07975
Epoch: 200, loss: 0.00785
Epoch: 250, loss: 0.00405
Epoch: 300, loss: 0.00307
Epoch: 350, loss: 0.00249
Epoch: 400, loss: 0.00199
Epoch: 450, loss: 0.00158


LSTM(
  (lstm): LSTM(2, 4, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [5]:
_ = lstm(testX) # Warm Start
test_predict = lstm(testX)
print("Test Predictions")
print(testY)
print(test_predict)

Test Predictions
tensor([0.0000, 1.0500, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000])
tensor([ 0.0808,  0.7560, -0.0124, -0.0010,  0.0210,  0.9736, -0.0322],
       grad_fn=<SelectBackward>)


In [6]:

import pandas as pd
pd.DataFrame({"train_actual":trainY.clone().detach().numpy(),"train_predicted":lstm(trainX).clone().detach().numpy()})


,train_actual,train_predicted
0,0.00,0.014708
1,1.01,0.984365
2,0.00,-0.016245
3,0.00,-0.000627
4,0.00,-0.006358
5,0.00,-0.072045
6,2.02,2.083875
7,0.00,0.020789
8,0.00,-0.008415
9,0.00,0.007827


In [80]:
list(zip(testY.clone().detach().numpy(),lstm(testX).clone().detach().numpy()))

[(0.0, 0.14097069),
 (1.05, 0.9100486),
 (0.0, -0.011163048),
 (0.0, -0.0036913007),
 (0.0, 0.042710528),
 (1.0, 1.0090622),
 (0.0, 0.27711207)]

In [81]:
for tup in list(zip(trainX.clone().detach().numpy(),lstm(trainX).clone().detach().numpy())):
    print(tup)
    print("-"*50)

(array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 1.01]], dtype=float32), 0.018263802)
--------------------------------------------------
(array([[0.  , 0.  ],
       [0.  , 1.01],
       [0.  , 0.  ]], dtype=float32), 1.0180054)
--------------------------------------------------
(array([[0.  , 1.01],
       [0.  , 0.  ],
       [1.01, 0.  ]], dtype=float32), -0.021573275)
--------------------------------------------------
(array([[0.  , 0.  ],
       [1.01, 0.  ],
       [0.  , 0.  ]], dtype=float32), -0.0034626275)
--------------------------------------------------
(array([[1.01, 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ]], dtype=float32), 0.004137352)
--------------------------------------------------
(array([[0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 2.02]], dtype=float32), -0.09415157)
--------------------------------------------------
(array([[0.  , 0.  ],
       [0.  , 2.02],
       [0.  , 0.  ]], dtype=float32), 2.0253365)
-------------------------------------

In [82]:
for tup in list(zip(testX.clone().detach().numpy(),lstm(testX).clone().detach().numpy())):
    print(tup)
    print("-"*50)

(array([[0.  , 0.  ],
       [2.01, 0.  ],
       [0.  , 1.05]], dtype=float32), 0.14097069)
--------------------------------------------------
(array([[2.01, 0.  ],
       [0.  , 1.05],
       [0.  , 0.  ]], dtype=float32), 0.9100486)
--------------------------------------------------
(array([[0.  , 1.05],
       [0.  , 0.  ],
       [1.05, 0.  ]], dtype=float32), -0.011163048)
--------------------------------------------------
(array([[0.  , 0.  ],
       [1.05, 0.  ],
       [0.  , 0.  ]], dtype=float32), -0.0036913007)
--------------------------------------------------
(array([[1.05, 0.  ],
       [0.  , 0.  ],
       [0.  , 1.  ]], dtype=float32), 0.042710528)
--------------------------------------------------
(array([[0., 0.],
       [0., 1.],
       [0., 0.]], dtype=float32), 1.0090622)
--------------------------------------------------
(array([[0. , 1. ],
       [0. , 0. ],
       [1. , 1.1]], dtype=float32), 0.27711207)
--------------------------------------------------


196